In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

data = xr.open_dataset("1_data_processing/1_1_Forcing/products/all_stations_cmems.zarr", engine="zarr")
data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
data.time.attrs["axis"] = "T"
data = data.sel(time=slice(time_start, time_end))
_ = data.load()

cafe_npp = xr.open_dataset("1_data_processing/1_1_Forcing/products/all_stations_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time", how="all")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.time.attrs["axis"] = "T"
_ = cafe_npp.load()

In [3]:
data

<xarray.Dataset> Size: 13MB
Dimensions:              (latitude: 3, longitude: 3, time: 8767, depth: 3)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 24B 22.75 31.6 50.0
  * longitude            (longitude) float64 24B -158.0 -150.0 -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 2MB 23.52...
    U                    (latitude, longitude, time, depth) float64 2MB 0.016...
    V                    (latitude, longitude, time, depth) float64 2MB -0.07...
    mnkc_epi             (latitude, longitude, time) float64 631kB 0.269 ... nan
    mnkc_hmlmeso         (latitude, longitude, time) float64 631kB 0.7915 ......
    mnkc_lmeso           (latitude, longitude, time) float64 631kB 2.173 ... nan
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 631kB 0.4889 ......
    mnkc_umeso           (latitude, longitude, time) float64 631kB 0.4931 ......
    npp                  (latitude, longitude, time) float64 631kB 315.7 ... nan
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 2MB 117.3...
    zeu                  (latitude, longitude, time) float64 631kB 78.19 ... nan
    zooc                 (latitude, longitude, time) float64 631kB 0.2896 ......

In [4]:
cafe_npp

<xarray.Dataset> Size: 701kB
Dimensions:    (time: 8767, latitude: 3, longitude: 3)
Coordinates:
  * latitude   (latitude) float64 24B 22.75 31.6 50.0
  * longitude  (longitude) float64 24B -158.0 -150.0 -64.2
  * time       (time) datetime64[ns] 70kB 1998-01-01 1998-01-02 ... 2022-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 631kB 306.4 nan ... nan nan
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [5]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("1_data_processing/1_1_Forcing/products/Bats_obs.zarr", engine="zarr")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)

obs_hot = xr.open_dataset("1_data_processing/1_1_Forcing/products/Hot_obs.zarr", engine="zarr")
obs_hot = update_layer(obs_hot, epipelagic_size)
obs_hot = Observation(obs_hot)

obs_papa = xr.open_dataset("1_data_processing/1_1_Forcing/products/Papa_obs.zarr", engine="zarr")
obs_papa = update_layer(obs_papa, epipelagic_size)
obs_papa = Observation(obs_papa)

observations = [obs_bats, obs_hot, obs_papa]

Create structure for SeapoPym simulation.


In [6]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=data["T"], resolution=1 / 12, timestep=1),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12, timestep=1),
)

2024-12-04 17:29:56,314 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [7]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        # tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        # tr_max=Parameter("D1N1_tr_max", 0, 50),
        tr_rate=-0.11,
        tr_max=10.38,
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        # tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        # tr_max=Parameter("D2N1_tr_max", 0, 50),
        tr_rate=-0.11,
        tr_max=10.38,
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [8]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [9]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    MUTPB=0.1,
    INDPB=0.2,
    ETA=5,
    CXPB=0.8,
    NGEN=3,
    POP_SIZE=10,
    cost_function_weight=(-(1 / 3), -(1 / 3), -(1 / 3)),
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [10]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [11]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

And watch the magic on the Dask dashboard :


In [12]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52382,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52396,Total threads: 2
Dashboard: http://127.0.0.1:52400/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52385,


And execute the process.


In [13]:
viewer = genetic_algo.optimize()

['D1N1_inv_lambda_max', 'D2N1_inv_lambda_rate', 'D1N1_inv_lambda_rate', 'D1N1_energy_coefficient', 'D2N1_energy_coefficient', 'D2N1_inv_lambda_max']


Finaly here is the result :


In [15]:
viewer.stats

,mean,std,min,max,valid,generation_gap
generation,,,,,,


In [16]:
viewer.logbook

D1N1_energy_coefficient  \
generation previous_generation individual                            
0          False               0                          0.585693   
                               1                          0.604000   
                               2                          0.290086   
                               3                          0.744433   
                               4                          0.535499   
                               5                          0.436188   
                               6                          0.227117   
                               7                          0.324647   
                               8                          0.362587   
                               9                          0.586581   
1          False               0                          0.744433   
                               1                          0.436188   
                               2                          0.535499   
                               3                          0.290086   
                               4                          0.324647   
                               8                          0.535499   
                               9                          0.227117   
           True                5                          0.436188   
                               6                          0.585693   
                               7                          0.227117   
2          False               0                          0.535499   
                               1                          0.227117   
                               2                          0.324647   
                               3                          0.744433   
                               4                          0.744433   
                               5                          0.227117   
                               6                          0.535499   
                               7                          0.585693   
                               8                          0.535499   
                               9                          0.324647   
3          False               0                          0.744433   
                               1                          0.324647   
                               2                          0.535499   
                               3                          0.227117   
                               4                          0.535499   
                               5                          0.324647   
                               8                          0.324647   
                               9                          0.585693   
           True                6                          0.324647   
                               7                          0.324647   

                                           D1N1_inv_lambda_max  \
generation previous_generation individual                        
0          False               0                     28.807048   
                               1                     50.456188   
                               2                     47.330161   
                               3                     88.340086   
                               4                     95.529061   
                               5                      3.890225   
                               6                     45.449041   
                               7                     53.478412   
                               8                     48.387793   
                               9                     23.931084   
1          False               0                      3.890225   
                               1                     88.340086   
                               2                     95.529061   
                               3                     47.330161   
                               4              

In [17]:
viewer.hall_of_fame

,,D1N1_energy_coefficient,D1N1_inv_lambda_max,D1N1_inv_lambda_rate,D2N1_energy_coefficient,D2N1_inv_lambda_max,D2N1_inv_lambda_rate,fitness
generation,individual,,,,,,,


In [18]:
viewer.fitness_evolution()

In [19]:
viewer.box_plot(3)

In [20]:
fig = viewer.parallel_coordinates(
    4000,
    # colorscale=[
    #     [0, "rgba(0, 0, 255, 0.8)"],
    #     [0.3, "rgba(255,255,0,0.5)"],
    #     [1.0, "rgba(255,255,0,0.0)"],
    # ],
)
fig.update_layout(width=1000, height=700)

In [21]:
from seapopym_optimization import wrapper

model_print = wrapper.model_generator_no_transport(
    forcing_parameters=forcing_parameters,
    fg_parameters=wrapper.FunctionalGroupGeneratorNoTransport(
        [
            [10.38, -0.11, 72, 0.09, 1, 1, 0.1],
            [10.38, -0.11, 44, 0.18, 2, 1, 0.46],
        ],
        ["D1N1", "D2N1"],
    ),
)

model_print.run()
biomass_print = model_print.export_biomass()

2024-12-04 17:30:07,684 :: Seapodym ::  WARNING ::
|	Parameter temperature_recruitment_rate : 1.0 has a positive value. It means that the recruitment time is increasing with temperature. Do you mean to use a negative value?



ValueError: 'energy_transfert' must be <= 1: 72.0

In [ ]:
obs_hot.observation.day.pint.quantify().pint.to("g/m2").plot()
biomass_print.sel(latitude=22.75, longitude=-158, functional_group=0).pint.quantify().pint.to("g/m2").plot()